In [ ]:
# !pip3 install rembg
# !pip3 install torch==1.7.1+cpu torchvision==0.8.2+cpu -f https://download.pytorch.org/whl/torch_stable.html
# !pip3 install scikit-image
# !pip3 install opencv-python
# !pip3 install np
# !pip3 install openvino

In [9]:
import cv2
import numpy as np
from rembg import remove
from inference import Create3DModel

In [10]:
video_source = "./vid.mp4"
image_save = "./image.png"
mask_save = "./mask.png"
model_save = "./model.obj"    

In [11]:
def GenerateMask(img):
    mask = (img[:,:,3] > 0) * np.uint8(1)
    mask = cv2.erode(mask,  cv2.getStructuringElement(cv2.MORPH_RECT, (3,3),(1,1)))
    mask = cv2.dilate(mask,  cv2.getStructuringElement(cv2.MORPH_RECT, (3,3),(1,1)))
    return mask
    
def Padding(size,img):
    if(len(img.shape) > 2):
        size = (size[0],size[1],img.shape[2])
    ret = np.zeros(size,dtype=np.uint8)
    ratio = img.shape[0]/img.shape[1]
    if(ratio > 0):
        img = cv2.resize(img,(int(ret.shape[0]/ratio),ret.shape[0]))
    else:
        img = cv2.resize(img,(ret.shape[1],int(ret.shape[1]*ratio)))
    c_y=ret.shape[0]//2
    c_x=ret.shape[1]//2
    _c_y = img.shape[0]//2
    _c_x = img.shape[1]//2
    ret[c_y-_c_y:c_y+img.shape[0]-_c_y,c_x-_c_x:c_x+img.shape[1]-_c_x] = img[:,:]
    # for y in range(img.shape[0]):
    #     for x in range(img.shape[1]):
    #         ret[c_y+(y-_c_y),c_x+(x-_c_x)] = img[y,x]
    return ret

In [12]:
cam = cv2.VideoCapture(video_source)
vw  = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
vh = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [13]:
body_outline_source = cv2.imread("body_outline.png",cv2.IMREAD_GRAYSCALE)
body_outline_ratio = body_outline_source.shape[1]/body_outline_source.shape[0]
body_outline_source = cv2.resize(
    body_outline_source,
    (int(body_outline_ratio*vh),vh),
    cv2.INTER_AREA
)

In [14]:
video_width_center = vw//2
body_width_center =  body_outline_source.shape[1]//2

In [15]:
body_outline_mask = np.ones((vh,vw,3),dtype=np.uint8)
for y in range(0,body_outline_source.shape[0],1):
    for x in range(0,body_outline_source.shape[1],1):
        if(body_outline_source[y,x]!=255):
            body_outline_mask[y,video_width_center + (x - body_width_center),0] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),1] = 0
            body_outline_mask[y,video_width_center + (x - body_width_center),2] = 0
body_outline_mask = Padding((512,512),body_outline_mask)
body_outline_indicator = (1-body_outline_mask) * 255

In [16]:
cv2.namedWindow("Camera Window")
img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break

    k = cv2.waitKey(1)
    if k%256 == 32:
        # SPACE pressed
        img = remove(frame)
        img =  Padding((512,512),img)
        mask =  Padding((512,512),GenerateMask(img)*np.uint8(255))
        cv2.imwrite(image_save,img)
        cv2.imwrite(mask_save,mask)
        Create3DModel(image_save,mask_save,model_save)
    else:
        frame = Padding((512,512),frame)
        frame = np.multiply(frame,body_outline_mask) + body_outline_indicator
        cv2.imshow("Camera Window", frame)
cam.release()
cv2.destroyAllWindows()

Init OpenVINO....
Start to 3D reconstruction...
Saving model ...
Time Cost:  29.43693350000001
failed to grab frame
